In [1]:
# @Author: Apeksha
# Approach: Using sklearn MultiOutputClassifier generating multiple label output.
# Step: 1. Read the processed label and features.
#       2. Apply PCA, REG, Normalization, Scaling
#       3. Prepare data for Cross Validation 
#       4. Do Classification
#       5. Evaluation

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,matthews_corrcoef
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

In [7]:
#Method to read dataset
def read_dataset(path):
    return pd.read_csv(path,header="infer",sep=",")

In [ ]:
#set path to read the processed features files
features_path="features.csv"

#set path to read processed label file
label_path="labels.csv"

feature_df=read_dataset(features_path);
label_df=read_dataset(label_path);

In [ ]:
#Add code for cross validation if there is anyother way of doing it.
X_train, X_test, y_train, y_test=train_test_split(feature_df,label_df, test_size=0.2,random_state=11) 

In [12]:
# Method to train the model
# Params- Estimator: Classifier being used for classification
#         X_train: input features
#         Y_train: input labels
#Return- Trained model of type Estimator 
def fit_model(estimator,X_train, y_train):
    return MultiOutputClassifier(estimator).fit(X_train, y_train)

# Method to make the predictions
# Params- Model: Trained model
#         X_test: Test data
#Return-  A list of labels per record. [[],[],[],[]] of shape 82117 * 27
def predict(model,X_test):
    return model.predict(X_test)

# Method to get model evaluation paramaters
def print_classification_report(predictions):
    print(classification_report(y_test, predictions))
    
# Method to write predictions to a text file.
def generate_output_file(predictions):
    output_path="output.txt"
    with open(output_path, 'w') as filehandle:
        filehandle.writelines("%s\n" % label for label in predictions)

In [ ]:
#Logistic regression Estimator
lr_estimator=LogisticRegression()
#C=1.0, class_weight=None, dual=False, fit_intercept=True,intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,       penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
#          verbose=0, warm_start=False

model=fit_model(lr_estimator,X_train, y_train)
predictions=model.predict(X_test)

print_classification_report(predictions)

In [ ]:
# Random Forest Estimator
rf_estimator=RandomForestClassifier(n_estimators=100,bootstrap = True,max_features = 'sqrt')
#rf_estimator = RandomForestClassifier(n_estimators=100) #Submission one
#rf_estimator = RandomForestClassifier(bootstrap=False,n_estimators=100,max_features=None,class_weight="balanced") #Submission 2 and 3

model=fit_model(rf_estimator,X_train, y_train)
predictions=model.predict(X_test)

print_classification_report(predictions)

In [ ]:
# SVM Estimator
sv_estimator=SVC(C=100,gamma=1,kernel="sigmoid")

model=fit_model(sv_estimator,X_train, y_train)
predictions=model.predict(X_test)

print_classification_report(predictions)